In [ ]:
import htcondor
import ipywidgets as widgets
from IPython.display import display
import os

# ─── 1) Script field (user enters or pastes their path) ────────────
script = widgets.Text(
    value='',
    description='Script path:',
    placeholder='/home/you/project/my_job.sh',
)
script.style.description_width = '30%'
script.layout = widgets.Layout(width='60%')

# ─── 2) Other inputs ────────────────────────────────────────────────
args = widgets.Text(
    value='',
    description='Arguments:',
)
args.style.description_width = '30%'
args.layout = widgets.Layout(width='60%')

cpus = widgets.IntSlider(
    value=1, min=1, max=16,
    description='CPUs:'
)
cpus.style.description_width = '30%'
cpus.layout = widgets.Layout(width='60%')

mem = widgets.IntText(
    value=1024,
    description='Memory (MB):'
)
mem.style.description_width = '30%'
mem.layout = widgets.Layout(width='60%')

submit_btn = widgets.Button(description='Submit Job', button_style='primary')
status_out = widgets.Output()

# ─── 3) Status box ───────────────────────────────────────────────────
status_box = widgets.VBox([status_out])
status_box.layout = widgets.Layout(
    width='60%',
    min_height='4rem',
    margin='1rem 0 0 0',
    border='1px solid #ddd',
    padding='0.5rem',
    border_radius='4px'
)

# ─── 4) Assemble & center the card ──────────────────────────────────
card = widgets.VBox(
    [
        widgets.HTML("<h1 style='text-align:center;'>HTCondor Submission</h1>"),
        widgets.HTML("<p style='text-align:center;color:#555;'>Enter the values relevant to your job, then submit</p>"),
        script, args, cpus, mem,
        submit_btn,
        status_box
    ],
    layout=widgets.Layout(
        width='80%', max_width='800px', padding='2%',
        border='1px solid #ddd', border_radius='8px',
        display='flex', flex_flow='column', align_items='center'
    )
)

container = widgets.Box(
    [card],
    layout=widgets.Layout(
        display='flex', align_items='center',
        justify_content='center', height='100vh', width='100vw'
    )
)

display(container)

# ─── 5) Submission handler ───────────────────────────────────────────
def on_submit(b):
    b.disabled = True
    status_out.clear_output()
    with status_out:
        print("🚀 Queuing…")
    try:
        path = script.value.strip()
        if not path:
            raise FileNotFoundError("No script path provided.")
        if not os.path.isabs(path):
            # interpret relative to current dir
            path = os.path.abspath(path)
        if not os.path.exists(path):
            raise FileNotFoundError(f"`{path}` not found.")
        schedd = htcondor.Schedd()
        desc = htcondor.Submit({
            'executable': path,
            'arguments': args.value,
            'request_cpus': str(cpus.value),
            'request_memory': f"{mem.value}MB",
            'output': '$(Cluster).$(Process).out',
            'error':  '$(Cluster).$(Process).err',
            'log':    '$(Cluster).log'
        })
        with schedd.transaction() as txn:
            cid = desc.queue(txn)
        with status_out:
            print(f"✅ Submitted as cluster {cid}")
    except Exception as e:
        with status_out:
            print(f"❌ Failed: {e}")
    finally:
        b.disabled = False

submit_btn.on_click(on_submit)


